In [1]:
import sys
!{sys.executable} -m pip install optuna

import numpy as np
import optuna
import torch
from torch import nn
from torchvision.models import resnet18, AlexNet
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import Subset
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, Lambda
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 209 kB 65.7 MB/s 
     |████████████████████████████████| 81 kB 13.2 MB/s 
     |████████████████████████████████| 78 kB 9.6 MB/s 
     |████████████████████████████████| 146 kB 67.4 MB/s 
     |████████████████████████████████| 112 kB 75.5 MB/s 
     |████████████████████████████████| 49 kB 7.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=beea1af1ad7632a6000f7dffe71f35b0770dadf1b8a8134d92556caf66976a03
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def get_model(model_name, num_classes):
    if model_name == "resnet":
      model = resnet18(num_classes=num_classes)
      model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) 
      return model
    elif model_name == "alexnet":
      model = AlexNet(num_classes)
      model.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
      return model
    else:
      raise ValueError("Unknown model name")

In [4]:
# Load dataset and prepare the dataloader
def get_dataloaders(dataset_name, batch_size, subset_ratio):
    if dataset_name == "mnist":
        dataset = datasets.MNIST(download=True, train=True, root="data").train_data.float()
    elif dataset_name == "fashion_mnist":
        dataset = datasets.FashionMNIST(download=True, train=True, root="data").train_data.float()
    else:
        raise ValueError("Unknown dataset name")
    data_transform = Compose([ Resize((224, 224)),ToTensor(), Normalize((dataset.mean()/255,), (dataset.std()/255,))])

    if dataset_name == "mnist":
        training_data = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=data_transform)
        
        test_data = datasets.MNIST(
        root="data",
        train=False,
        download=True,
        transform=data_transform)
    elif dataset_name == "fashion_mnist":
        training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=data_transform)
        
        test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=data_transform)
    else:
        raise ValueError("Unknown dataset name")

    trainset = Subset(training_data, list(range(0, len(training_data), subset_ratio)))
    testset = Subset(test_data, list(range(0, len(test_data), subset_ratio)))
    
    train_dataloader = DataLoader(trainset, batch_size=batch_size)
    test_dataloader = DataLoader(testset, batch_size=batch_size)
    
    return train_dataloader, test_dataloader
    

In [5]:
# Training loop. Source : https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
def train(model, train_loader, loss_function, optimizer):
    model.train()
    size = len(train_loader.dataset)
        
    for i, data in enumerate(train_loader):
        X,y = data
        X = X.to(device)
        y = y.to(device)
            
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            loss, current = loss.item(), i * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")        

In [6]:
# Validation loop : Source : https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
def validate(model, dataloader, loss_fn):
    model.eval()
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return correct

In [17]:
model_name = ""
dataset_name = ""

def objective(trial):
    dataset_subset = 60
    num_classes = 10
    epochs = 5
    model = get_model(model_name, num_classes)
    batch_size = 2 ** trial.suggest_int('batch size 2^i', 0, 7)
    optimizer = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])
    loss_function = torch.nn.CrossEntropyLoss()
    learning_rate = trial.suggest_float('learning rate', 1e-5, 1e-1, log=True)
    
    train_dataloader, test_dataloader = get_dataloaders(dataset_name, batch_size, dataset_subset)
    
    if optimizer == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    model.to(device)
    
    for epoch in range(epochs):
      train(model, train_dataloader, loss_function, optimizer)
      validate(model, test_dataloader, loss_function)
        
    return validate(model, test_dataloader, loss_function)

In [18]:
# Create study and launch study
model_name = "alexnet"
dataset_name = "mnist"
study1 = optuna.create_study(direction="maximize")
study1.optimize(objective, n_trials=100)

[I 2022-06-16 19:10:18,150] A new study created in memory with name: no-name-bd920c5b-2fcf-486c-a004-b8df3c5aa864


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



...

Test Error: 
 Accuracy: 54.5%, Avg loss: 1.316635 

Test Error: 
 Accuracy: 54.5%, Avg loss: 1.316635 



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning:

train_data has been renamed data



loss: 2.302730  [    0/ 1000]
Test Error: 
 Accuracy: 77.8%, Avg loss: 0.863655 

loss: 1.148118  [    0/ 1000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.313531 

loss: 0.504881  [    0/ 1000]
Test Error: 
 Accuracy: 92.8%, Avg loss: 0.156522 

loss: 0.181137  [    0/ 1000]
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.179731 

loss: 0.535880  [    0/ 1000]


[I 2022-06-16 19:46:36,941] Trial 99 finished with value: 0.9341317365269461 and parameters: {'batch size 2^i': 4, 'learning rate': 0.00014014621226986936}. Best is trial 48 with value: 0.9640718562874252.


Test Error: 
 Accuracy: 93.4%, Avg loss: 0.137503 

Test Error: 
 Accuracy: 93.4%, Avg loss: 0.137503 



In [19]:
study1.best_params

{'batch size 2^i': 4, 'learning rate': 0.00018671851347899118}

In [20]:
optuna.visualization.plot_optimization_history(study1)

In [21]:
optuna.visualization.plot_contour(study1)

In [22]:
optuna.visualization.plot_param_importances(study1)

In [23]:
optuna.visualization.plot_contour(study1, ["learning rate", "batch size 2^i"])

In [ ]:
optuna.visualization.plot_contour(study1, ["learning rate", "optimizer"])

In [ ]:
model_name = "alexnet"
dataset_name = "fashion_mnist"
study2 = optuna.create_study(direction="maximize")
study2.optimize(objective, n_trials=100)

[I 2022-06-16 19:49:43,845] A new study created in memory with name: no-name-9213914f-ea82-4e64-a67a-ad5c4f35c41f
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning:

train_data has been renamed data



loss: 2.285721  [    0/ 1000]
loss: 2.264567  [  100/ 1000]
loss: 2.308059  [  200/ 1000]
loss: 2.625175  [  300/ 1000]
loss: 0.004661  [  400/ 1000]
loss: 1.989603  [  500/ 1000]
loss: 1.524425  [  600/ 1000]
loss: 0.610551  [  700/ 1000]
loss: 1.291685  [  800/ 1000]
loss: 0.130740  [  900/ 1000]
Test Error: 
 Accuracy: 59.9%, Avg loss: 0.901258 

loss: 0.009417  [    0/ 1000]
loss: 2.533892  [  100/ 1000]
loss: 1.550967  [  200/ 1000]
loss: 3.094552  [  300/ 1000]
loss: 0.023310  [  400/ 1000]
loss: 0.556010  [  500/ 1000]
loss: 1.630195  [  600/ 1000]
loss: 0.244999  [  700/ 1000]
loss: 0.000000  [  800/ 1000]
loss: 0.180364  [  900/ 1000]
Test Error: 
 Accuracy: 72.5%, Avg loss: 0.749179 

loss: 0.007480  [    0/ 1000]
loss: 2.538554  [  100/ 1000]
loss: 1.078369  [  200/ 1000]
loss: 1.657420  [  300/ 1000]
loss: 0.252429  [  400/ 1000]
loss: 0.106551  [  500/ 1000]
loss: 1.199074  [  600/ 1000]
loss: 1.835453  [  700/ 1000]
loss: 0.000000  [  800/ 1000]
loss: 0.092173  [  900/ 10

[I 2022-06-16 19:52:05,729] Trial 0 finished with value: 0.7425149700598802 and parameters: {'batch size 2^i': 0, 'learning rate': 0.00010017451410412927}. Best is trial 0 with value: 0.7425149700598802.


Test Error: 
 Accuracy: 74.3%, Avg loss: 0.794098 



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning:

train_data has been renamed data



...

loss: 2.297437  [    0/ 1000]
loss: 2.265129  [  100/ 1000]
loss: 2.194321  [  200/ 1000]
loss: 2.428440  [  300/ 1000]
loss: 0.015493  [  400/ 1000]
loss: 2.871642  [  500/ 1000]
loss: 1.358933  [  600/ 1000]
loss: 0.530087  [  700/ 1000]
loss: 0.248216  [  800/ 1000]
loss: 0.260048  [  900/ 1000]
Test Error: 
 Accuracy: 62.9%, Avg loss: 0.896314 

loss: 0.058125  [    0/ 1000]
loss: 1.633589  [  100/ 1000]
loss: 1.767425  [  200/ 1000]
loss: 4.466691  [  300/ 1000]
loss: 0.003643  [  400/ 1000]
loss: 0.346823  [  500/ 1000]
loss: 0.761044  [  600/ 1000]
loss: 0.592185  [  700/ 1000]
loss: 0.000009  [  800/ 1000]
loss: 0.381549  [  900/ 1000]
Test Error: 
 Accuracy: 70.7%, Avg loss: 0.709085 

loss: 0.008274  [    0/ 1000]
loss: 3.926125  [  100/ 1000]
loss: 1.290109  [  200/ 1000]
loss: 1.985429  [  300/ 1000]
loss: 0.002893  [  400/ 1000]
loss: 0.168312  [  500/ 1000]
loss: 1.318686  [  600/ 1000]
loss: 3.065871  [  700/ 1000]
loss: 0.000000  [  800/ 1000]
loss: 0.247218  [  900/ 10

[I 2022-06-16 21:22:40,359] Trial 83 finished with value: 0.7544910179640718 and parameters: {'batch size 2^i': 0, 'learning rate': 0.00014836227171988745}. Best is trial 43 with value: 0.8023952095808383.


Test Error: 
 Accuracy: 75.4%, Avg loss: 0.635140 



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning:

train_data has been renamed data



loss: 2.298114  [    0/ 1000]
loss: 2.080734  [  200/ 1000]
loss: 0.627248  [  400/ 1000]
loss: 1.661973  [  600/ 1000]
loss: 1.013667  [  800/ 1000]
Test Error: 
 Accuracy: 67.7%, Avg loss: 0.890167 

loss: 0.238744  [    0/ 1000]
loss: 0.734233  [  200/ 1000]
loss: 0.692374  [  400/ 1000]


In [ ]:
study2.best_params

In [ ]:
optuna.visualization.plot_optimization_history(study2)

In [ ]:
optuna.visualization.plot_contour(study2)

In [ ]:
optuna.visualization.plot_param_importances(study2)

In [16]:
optuna.visualization.plot_contour(study2, ["learning rate", "batch size 2^i"])

NameError: ignored

In [14]:
optuna.visualization.plot_contour(study2, ["learning rate", "optimizer"])

In [ ]:
model_name = "resnet"
dataset_name = "mnist"
study3 = optuna.create_study(direction="maximize")
study3.optimize(objective, n_trials=100)

[I 2022-06-15 18:01:28,691] A new study created in memory with name: no-name-c3c84ed3-52e3-4425-a596-49c27e4ecd33


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



...

Test Error: 
 Accuracy: 98.2%, Avg loss: 0.088432 



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


loss: 2.425256  [    0/ 1000]
loss: 0.477733  [  800/ 1000]
Test Error: 
 Accuracy: 90.4%, Avg loss: 0.354610 

loss: 0.670904  [    0/ 1000]
loss: 0.236160  [  800/ 1000]
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.323550 

loss: 0.282982  [    0/ 1000]
loss: 0.089548  [  800/ 1000]
Test Error: 
 Accuracy: 93.4%, Avg loss: 0.215857 

loss: 0.217144  [    0/ 1000]
loss: 0.041748  [  800/ 1000]
Test Error: 
 Accuracy: 83.8%, Avg loss: 0.475936 

loss: 0.159648  [    0/ 1000]
loss: 0.021089  [  800/ 1000]
Test Error: 
 Accuracy: 95.8%, Avg loss: 0.133551 



[I 2022-06-15 18:45:14,095] Trial 99 finished with value: 0.9580838323353293 and parameters: {'batch size 2^i': 3, 'optimizer': 'Adam', 'learning rate': 0.00020208733830024472}. Best is trial 98 with value: 0.9820359281437125.


Test Error: 
 Accuracy: 95.8%, Avg loss: 0.133551 



In [ ]:
study3.best_params

{'batch size 2^i': 3,
 'learning rate': 0.0002299351407789967,
 'optimizer': 'Adam'}

In [ ]:
study4.best_params

NameError: ignored

In [ ]:
optuna.visualization.plot_optimization_history(study3)

In [ ]:
optuna.visualization.plot_contour(study3)

In [ ]:
optuna.visualization.plot_param_importances(study3)

In [ ]:
model_name = "resnet"
dataset_name = "fashion_mnist"
study4 = optuna.create_study(direction="maximize")
study4.optimize(objective, n_trials=100)

[I 2022-06-15 18:45:14,845] A new study created in memory with name: no-name-11db549c-3e4d-4867-affa-aef5beecb281


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



...

Test Error: 
 Accuracy: 72.5%, Avg loss: 0.793787 



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning:

train_data has been renamed data



loss: 2.624551  [    0/ 1000]
loss: 1.240839  [  400/ 1000]
loss: 1.085011  [  800/ 1000]
Test Error: 
 Accuracy: 60.5%, Avg loss: 0.977041 

loss: 0.630207  [    0/ 1000]
loss: 0.543493  [  400/ 1000]
loss: 0.749991  [  800/ 1000]
Test Error: 
 Accuracy: 67.7%, Avg loss: 0.781262 

loss: 0.341757  [    0/ 1000]
loss: 0.481867  [  400/ 1000]
loss: 0.359464  [  800/ 1000]
Test Error: 
 Accuracy: 72.5%, Avg loss: 0.705308 

loss: 0.221287  [    0/ 1000]
loss: 0.285290  [  400/ 1000]
loss: 0.103985  [  800/ 1000]
Test Error: 
 Accuracy: 75.4%, Avg loss: 0.731359 

loss: 0.100018  [    0/ 1000]
loss: 0.049414  [  400/ 1000]
loss: 0.051011  [  800/ 1000]
Test Error: 
 Accuracy: 74.9%, Avg loss: 0.686222 



[I 2022-06-15 19:38:25,048] Trial 99 finished with value: 0.7485029940119761 and parameters: {'batch size 2^i': 2, 'optimizer': 'Adam', 'learning rate': 3.8691416469901486e-05}. Best is trial 55 with value: 0.7904191616766467.


Test Error: 
 Accuracy: 74.9%, Avg loss: 0.686222 



In [ ]:
optuna.visualization.plot_optimization_history(study4)

In [ ]:
optuna.visualization.plot_contour(study4)

In [ ]:
optuna.visualization.plot_param_importances(study4)

In [ ]:
optuna.visualization.plot_contour(study3, ["learning rate", "batch size 2^i"])

In [ ]:
# Example run
dataset_subset = 10
num_classes = 10
model = get_model(num_classes)
epochs = 10
batch_size = 32
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_function = torch.nn.CrossEntropyLoss()
    
train_dataloader, test_dataloader = get_dataloaders(batch_size, dataset_subset)

# Send to device
model.to(device)

for t in range(epochs):
    train(model, epochs, train_dataloader, loss_function, optimizer)
    validate(model, test_dataloader, loss_function)